# XSAR example

open a dataset with [xsar.open_dataset](../basic_api.rst#xsar.open_dataset)

In [ ]:
import xsar
import os
import numpy as np

In [ ]:
# get test file. You can replace with an path to other SAFE
filename = xsar.get_test_file('S1A_IW_GRDH_1SDV_20170907T103020_20170907T103045_018268_01EB76_Z200.SAFE')

## Open a dataset with a xsar.SentinelMeta object
A [xsar.SentinelMeta](../basic_api.rst#xsar.SentinelMeta) object handles all attributes and methods that can't be embdeded in a `xarray.Dataset` object.
It can also replace a filename in [xsar.open_dataset](../basic_api.rst#xsar.open_dataset)

When using it in a notebook, it also have an html representation.

In [ ]:
sar_meta = xsar.SentinelMeta(filename)
sar_meta

`sar_meta` object is an [xsar.SentinelMeta](../basic_api.rst#xsar.SentinelMeta) object that can be given to  [xsar.open_dataset](../basic_api.rst#xsar.open_dataset) , as if it was a filename:


In [ ]:
sar_ds = xsar.open_dataset(sar_meta)
sar_ds

## Open a dataset at lower resolution
We can use `sar_meta` properties to compute the pixel count we need to average to get a resolution of 200*200 meters

In [ ]:
resolution = {'atrack' : int(np.round(200 / sar_meta.pixel_atrack_m)), 'xtrack': int(np.round(200 / sar_meta.pixel_xtrack_m))}
resolution

In [ ]:
sar_ds = xsar.open_dataset(sar_meta, resolution=resolution)
sar_ds

## Extract a sub image of 10*10km around a lon/lat point

### Convert (lon,lat) to (atrack, xtrack)
we can use [sar_meta.ll2coords](../basic_api.rst#xsar.SentinelMeta.ll2coords) to convert (lon,lat) to (atrack, xtrack):

In [ ]:
# from a shapely object
point_lonlat =  sar_meta.footprint.centroid
point_coords = sar_meta.ll2coords(point_lonlat.x, point_lonlat.y)
point_coords

The result is floating, because it is the position inside the pixel.
If real indexes from existing dataset is needed, just pass the `dataset=` keyword. 
Result will be the nearest (atrack, xtrack) in the dataset

In [ ]:
point_coords = sar_meta.ll2coords(point_lonlat.x, point_lonlat.y, dataset=sar_ds)
point_coords

### Extract the sub-image

In [ ]:
box_size = 10000 # 10km
dist = {'atrack' : int(np.round(box_size / 2 / sar_meta.pixel_atrack_m)), 'xtrack': int(np.round(box_size / 2 / sar_meta.pixel_xtrack_m))}
dist

In [ ]:
# select 10*10 km around point_coords
sar_ds = sar_ds.sel(atrack=slice(point_coords[0] - dist['atrack'], point_coords[0] + dist['atrack']), xtrack=slice(point_coords[1] - dist['xtrack'], point_coords[1] + dist['xtrack']))
sar_ds